In [1]:
%%bash

# 增加更新
git add *.ipynb

git remote -v

git commit -m '更新 ch9 #1 change Jul 30, 2021'

git push origin master

origin	git@github.com:ustchope/dlwpn.git (fetch)
origin	git@github.com:ustchope/dlwpn.git (push)
[master fbec35bb] 更新 ch9 #1 change Jul 30, 2021
 3 files changed, 548 insertions(+), 223 deletions(-)
 create mode 100644 .ipynb_checkpoints/ch9-checkpoint.ipynb
 create mode 100644 ch9.ipynb


remote: warning: File feature_extraction_with_data_augmentation.keras is 81.20 MB; this is larger than GitHub's recommended maximum file size of 50.00 MB        
remote: error: GH001: Large files detected. You may want to try Git Large File Storage - https://git-lfs.github.com.        
remote: error: Trace: 132a741a01a9fcbbff7e8ff3a779d5e80b04db0aa1115563c765af6c95e6ae69        
remote: error: See http://git.io/iEPt8g for more information.        
remote: error: File fine_tuning.keras is 108.22 MB; this exceeds GitHub's file size limit of 100.00 MB        
To github.com:ustchope/dlwpn.git
 ! [remote rejected]   master -> master (pre-receive hook declined)
error: 推送一些引用到 'github.com:ustchope/dlwpn.git' 失败


CalledProcessError: Command 'b"\n# \xe5\xa2\x9e\xe5\x8a\xa0\xe6\x9b\xb4\xe6\x96\xb0\ngit add .\n\ngit remote -v\n\ngit commit -m '\xe6\x9b\xb4\xe6\x96\xb0 ch9 #1 change Jul 30, 2021'\n\ngit push origin master\n"' returned non-zero exit status 1.

In [ ]:
# 自动计算cell的计算时间
%load_ext autotime

#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU") 

In [ ]:
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

# 用于计算机视觉的高级深度学习

> 本章包含

* 计算机视觉的不同分支：图像分类、图像分割、对象检测
* 现代卷积网络架构模式：残差连接、批量归一化、深度可分离卷积
* 可视化和解释卷积神经网络所学内容的技术

前一章通过简单的模型（Conv2D 和 MaxPooling2D 层的堆栈）和一个简单的用例（二值图像分类）首次介绍了计算机视觉的深度学习。 但是计算机视觉不仅仅是图像分类！ 本章深入探讨更多样化的应用程序和高级最佳实践。

## 三个基本的计算机视觉任务

到目前为止，我们一直专注于图像分类模型：输入图像，输出标签。 “这张图片可能包含一只猫，另一张图片可能包含一只狗”。 但图像分类只是深度学习在计算机视觉中的几种可能应用之一。 一般来说，您需要了解三个基本的计算机视觉任务：
* **图像分类**——目标是为图像分配一个或多个标签。 它可以是单标签分类（一幅图像只能属于一个类别而排除其他类别），也可以是多标签分类（标记图像所属的所有类别，如图 9.1 所示）。 例如，当您在 Google 相册应用中搜索关键字时，您在后台查询的是一个非常大的多标签分类模型——该模型具有超过 20,000 个不同的类别，并在数百万张图像上进行训练。
* **图像分割**——目标是将图像“分割”或“划分”成不同的区域，每个区域通常代表一个类别（如图 9.1 所示）。 例如，当 Zoom 或 Google Meet 在视频通话中在您身后显示自定义背景时，它们会使用图像分割模型以像素精度区分您的脸及其背后的内容。
* **对象检测**——目标是在图像中感兴趣的对象周围绘制矩形（称为“边界框”），并将每个矩形与一个类相关联。 例如，自动驾驶汽车可以使用物体检测模型来监控汽车、行人和标志。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gsyv8jjfcoj313c0tgtjk.jpg)

除了这三个任务之外，计算机视觉的深度学习还包含一些更小众的任务，例如图像相似性评分（估计两个图像在视觉上的相似程度）、关键点检测（精确定位图像中感兴趣的属性，例如面部特征）、 姿态估计、3D 网格估计等。 但首先，图像分类、图像分割和对象检测构成了每个机器学习工程师都应该熟悉的基础。 大多数计算机视觉应用归结为这三者之一。

您已经在上一章中看到了图像分类的实际应用。 接下来，让我们深入研究图像分割。 这是一项非常有用且用途广泛的技术，您可以使用迄今为止所学的知识直接接触它。

请注意，我们不会涵盖对象检测，因为它对于一本介绍性书籍来说太专业且太复杂。 但是，您可以查看 Keras.io 上的 RetinaNet 示例，该示例展示了如何使用约 450 行代码在 Keras 中从头开始构建和训练异议检测模型。

## 图像分割示例

深度学习的图像分割是关于使用模型为图像中的每个像素分配一个类，从而将图像分割成不同的区域（例如“背景”和“前景”，或“道路”、“汽车”和“ 人行道”）。 这一通用类别的技术可用于为图像和视频编辑、自动驾驶、机器人技术、医学成像等领域的各种有价值的应用提供支持。

您应该了解两种不同风格的图像分割：
1. 语义分割，其中每个像素被独立地分类到一个语义类别中，如“猫”。 如果图像中有两只猫，则相应的像素都映射到相同的通用“猫”类别（见图 9.2）。
2. 实例分割，不仅要按类别对图像像素进行分类，还要解析出单个对象实例。 在其中有两只猫的图像中，实例分割会将“猫 1”和“猫 2”视为两个独立的像素类别（见图 9.2）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gsz3v3axnrj31fu0kudn2.jpg)

在这个例子中，我们将专注于语义分割：我们将再次查看猫和狗的图像，这次我们将学习区分主要主题及其背景。

我们将使用 Oxford-IIIT Pets 数据集 (www.robots.ox.ac.uk/~vgg/data/pets/)，其中
包含 7,390 张各种猫狗的图片，以及每张图片的前景-背景“分割蒙版”。 分割掩码是标签的图像分割等价物：它是与输入图像大小相同的图像，具有单个颜色通道，其中每个整数值对应于输入图像中相应像素的类别。 在我们的例子中，我们的分割掩码的像素可以采用三个整数值之一：
* 1（前景）
* 2（背景）
* 3（轮廓）

有关示例，请参见图 9.3。

让我们首先使用 wget和tar shell 实用程序下载和解压缩我们的数据集：

In [ ]:
%%bash

cd data

parallel wget ::: http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz \
::: http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz

In [ ]:
!tar -xf images.tar.gz
!tar -xf annotations.tar.gz

In [3]:
pwd

'/public/huangwei/test/deep-learning-with-python-notebooks'